1432. Max Difference You Can Get From Changing an Integer

In [ ]:
import csv
from typing import List, Dict, Optional
from collections import defaultdict

def anonymize_ids(
    ids: List[str],
    replacement_map: Dict[str, str],
    targeted_position: Optional[int] = None
) -> tuple[List[str], dict]:
    """
    Anonymize a list of numeric IDs by replacing digits based on a replacement map.
    
    Args:
        ids: List of numeric strings to anonymize
        replacement_map: Dictionary mapping original digits to replacement digits
        targeted_position: If set, only replace digits after this position (0-based index)
    
    Returns:
        Tuple of (anonymized IDs, statistics dictionary)
    """
    anonymized_ids = []
    stats = {
        'total_digits_changed': 0,
        'ids_affected': 0,
        'digit_change_counts': defaultdict(int)
    }
    
    for id_str in ids:
        original_id = id_str
        new_id = list(id_str)
        changed = False
        
        # Determine start position for replacement
        start_pos = targeted_position if targeted_position is not None else 0
        
        # Apply replacements
        for i in range(start_pos, len(id_str)):
            if id_str[i] in replacement_map:
                new_id[i] = replacement_map[id_str[i]]
                stats['digit_change_counts'][id_str[i] + '->' + new_id[i]] += 1
                stats['total_digits_changed'] += 1
                changed = True
                
        anonymized_id = ''.join(new_id)
        anonymized_ids.append(anonymized_id)
        if changed:
            stats['ids_affected'] += 1
            
    return anonymized_ids, stats

def process_csv_file(
    input_file: str,
    output_file: str,
    id_column: str,
    replacement_map: Dict[str, str],
    targeted_position: Optional[int] = None
) -> dict:
    """
    Process a CSV file to anonymize IDs in the specified column and save to a new file.
    
    Args:
        input_file: Path to input CSV file
        output_file: Path to output CSV file
        id_column: Name of the column containing IDs
        replacement_map: Dictionary mapping original digits to replacement digits
        targeted_position: If set, only replace digits after this position
    
    Returns:
        Statistics dictionary
    """
    # Read input CSV
    ids = []
    rows = []
    with open(input_file, 'r', newline='') as f:
        reader = csv.DictReader(f)
        if id_column not in reader.fieldnames:
            raise ValueError(f"Column '{id_column}' not found in CSV")
        
        for row in reader:
            ids.append(row[id_column])
            rows.append(row)
    
    # Anonymize IDs
    anonymized_ids, stats = anonymize_ids(ids, replacement_map, targeted_position)
    
    # Write output CSV
    with open(output_file, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=reader.fieldnames)
        writer.writeheader()
        for row, anon_id in zip(rows, anonymized_ids):
            row[id_column] = anon_id
            writer.writerow(row)
    
    return stats

# Example usage
def main():
    # Sample input
    sample_ids = ["12345", "23786", "33459"]
    replacement_map = {'3': '9', '2': '0'}
    
    # Test global replacement
    print("Global Replacement:")
    anon_ids, stats = anonymize_ids(sample_ids, replacement_map)
    print(f"Original IDs: {sample_ids}")
    print(f"Anonymized IDs: {anon_ids}")
    print(f"Stats: {stats}")
    
    # Test targeted replacement (skip first 2 digits)
    print("\nTargeted Replacement (after position 2):")
    anon_ids, stats = anonymize_ids(sample_ids, replacement_map, targeted_position=2)
    print(f"Original IDs: {sample_ids}")
    print(f"Anonymized IDs: {anon_ids}")
    print(f"Stats: {stats}")
    
    # Test CSV processing
    # Assuming input.csv exists with an 'id' column
    try:
        stats = process_csv_file(
            'input.csv',
            'output.csv',
            'id',
            replacement_map,
            targeted_position=2
        )
        print("\nCSV Processing Stats:")
        print(f"Stats: {stats}")
    except FileNotFoundError:
        print("\nCSV file not found. Please create input.csv with an 'id' column.")

if __name__ == "__main__":
    main()